In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.2.2'
# spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.c

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-11-22 23:45:51--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.79MB/s    in 0.2s    

2022-11-22 23:45:52 (5.79 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [6]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)

/content/spark-3.2.2-bin-hadoop2.7/python/pyspark/sql/context.py:79: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  FutureWarning


In [7]:
# Configuration for RDS instance
mode="append"
jdbc_url = "jdbc:postgresql://bootcampdemo.cjhuht28cggy.us-east-1.rds.amazonaws.com:5432/postgres"

config = {"user":"postgres",
          "password": "password",
          "driver":"org.postgresql.Driver"}

In [9]:
# Read Diebetes table struture from Postgres database on AWS

sqlContext.read.jdbc(url=jdbc_url, table="diabetes_data", properties=config)

DataFrame[diabetes_012: decimal(3,1), highbp: decimal(3,1), highchol: decimal(3,1), cholcheck: decimal(3,1), bmi: decimal(3,1), smoker: decimal(3,1), stroke: decimal(3,1), heartdiseaseorattack: decimal(3,1), physactivity: decimal(3,1), fruits: decimal(3,1), veggies: decimal(3,1), hvyalcoholconsump: decimal(3,1), anyhealthcare: decimal(3,1), nodocbccost: decimal(3,1), genhlth: decimal(3,1), menthlth: decimal(3,1), physhlth: decimal(3,1), diffwalk: decimal(3,1), sex: decimal(3,1), age: decimal(3,1), education: decimal(3,1), income: decimal(3,1)]

In [39]:
# Run SQL queries using pyspark

select_sql = "(select * from diabetes_data limit 100) as tbl"
df = sqlContext.read.jdbc(url=jdbc_url,table=select_sql, properties=config)
df.show(10)

+------------+------+--------+---------+----+------+------+--------------------+------------+------+-------+-----------------+-------------+-----------+-------+--------+--------+--------+---+----+---------+------+
|diabetes_012|highbp|highchol|cholcheck| bmi|smoker|stroke|heartdiseaseorattack|physactivity|fruits|veggies|hvyalcoholconsump|anyhealthcare|nodocbccost|genhlth|menthlth|physhlth|diffwalk|sex| age|education|income|
+------------+------+--------+---------+----+------+------+--------------------+------------+------+-------+-----------------+-------------+-----------+-------+--------+--------+--------+---+----+---------+------+
|         0.0|   1.0|     1.0|      1.0|40.0|   1.0|   0.0|                 0.0|         0.0|   0.0|    1.0|              0.0|          1.0|        0.0|    5.0|    18.0|    15.0|     1.0|0.0| 9.0|      4.0|   3.0|
|         0.0|   0.0|     0.0|      0.0|25.0|   1.0|   0.0|                 0.0|         1.0|   0.0|    0.0|              0.0|          0.0|    

In [40]:
# Run SQL queries using pyspark
# Males and Females NOT having good BMI index and count of no-diebetes, pre-diebetes, diebetes condition

select_sql = "(select diabetes_012, sex, count(*) as \"Total Count\" from diabetes_data where bmi not in (18, 19, 20, 21, 22, 23, 24, 25) group by diabetes_012, sex) as tbl"

df = sqlContext.read.jdbc(url=jdbc_url,table=select_sql, properties=config)
df.show(10)

+------------+---+-----------+
|diabetes_012|sex|Total Count|
+------------+---+-----------+
|         0.0|0.0|      68552|
|         0.0|1.0|      63650|
|         1.0|0.0|       2019|
|         1.0|1.0|       1673|
|         2.0|0.0|      15558|
|         2.0|1.0|      14326|
+------------+---+-----------+

